<a href="https://colab.research.google.com/github/minkyoJang/AI-BigDataStudy/blob/master/jmkDay23_CNN_self.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision #이미지 관련 처리, Pretrained Model 관련된 Package
import torchvision.datasets as vision_dsets
import torchvision.transforms as transforms #이미지 처리(vision)관련된 transformation
import torch.optim as optim #pytorch에서 정의한 수 많은 optimization function들 불러옵니다.
from torch.utils import data

import numpy as np
import matplotlib.pyplot as plt #시각화를 위한 패키지

In [0]:
device ='cuda' if torch.cuda.is_available() else 'cpu'
lr=0.001
batch_size= 128
best_acc=0 #best test accuracy
start_epoch=0 # start from epoch - or last checkpoint epoch

In [4]:
# torch.view 연습

##배치: 기존이 3차원이라고 하면, 배치를 사용하여 4차원으로 처리하게 함.
sample= torch.randn(2000,3,64,64) #2000배치, 3채널, 64하이트, 64윗드 //텐서플로는 순서가 다름
#코딩 시 B, C, H, W로 변수화 하여 사용하면 편리
# sample=> (2000,64,64,3)

B,C,H,W=sample.size()

#sample => (2000,64,64,3)
sample_1= sample.view(B,-1) #이 샘플을 (2000,)
print(sample_1.size())

torch.Size([2000, 12288])


In [7]:
sample_2= sample.view(B*C, H*W)
print(sample_2.size())

torch.Size([6000, 4096])


In [13]:
# torch.view 다른 예제
a= torch.FloatTensor(5,20,100) #1차원으로써 채널에 하이트만 있음
a1= a.view(2,10,-1)
print(a1.size())
a2= a.view(-1,10,200)
print(a2.size())

torch.Size([2, 10, 500])
torch.Size([5, 10, 200])


In [17]:
# dimension간에 이동을 하고 싶다면? ex) B, C, H, W => B, C, W, H > permute 사용
sample= torch.randn(2000,3,32,64)
sample_1=sample.permute(0,1,3,2)
print(sample.size())
print(sample_1.size())

torch.Size([2000, 3, 32, 64])
torch.Size([2000, 3, 64, 32])


In [21]:
# sample_1 변수의 디맨션을 원래대로 돌리려면?
sample_2= sample_1.permute(0,1,3,2)
print(torch.equal(sample,sample_2))

True


### Convolution layer  가지고 놀기 
- nn.Conv2d(in, out, filter_size, stride, padding)
- 각 이미지를 만들때 몇 개의 레이어를 쌓아서 만드는 것
- 커널=필터
- batch: 고양이 사진을 판별한다고 한다면, 사용하는 사진의 갯수(우진님)
    - 배치는 그대로
- padding 사용이유: 만일 스트라이드를 하면서 필터를 하다보면 점점 이미지가 사이즈가 작아지게 됨. 그러면 다음번 이미지 컨부하면서 원래의 데이터보다 작아지게 되니 이를 방지하게 위해 패딩으로 원 크기를 유지시키는 것 (석진 조교님)
- width, height는 그 공식 (input+2p-k/strids)+1

In [22]:
# torch.nn. conv 연습
# 클래스 구성: nn.Conv2d(in, out, filter_size, stride, padding)
# 아웃풋 데이터가 다음과 같이 되도록 해봅시다(16,128,32,32)

input1 = torch.zeros(16,3,64,64) 
conv1 = nn.Conv2d(3,512, 3,1,1 ) #16,3,64,64> 16, 512, 64,64
# 3,1,1 쓰면 아웃풋이 그대로 나온대. 커널로 5를 쓰면 패딩을 2로주고 스페셜 그대로 유지
conv2= nn.Conv2d(512,128, 3,2,1) #16, 512, 64, 64 > 16,128,32,32
print(conv2)

out= conv1(input1)
output=conv2(out)
print(output.size())

Conv2d(512, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
torch.Size([16, 128, 32, 32])


# 데이터 로딩
- Data Augmentation
    - 이미지 하나를 트레이닝 시키려면 여러개의 소스가 필요한데 실질적으로 그렇게 많은 소스를 가져올 수 없음
    - 따라서 하나의 이미지를 회전하는 식으로 데이터 소스를 얻습니다.

In [29]:
# 데이터 로딩: 
print('==> Preparing data..')

#데이터 전처리를 위한 코드
transform_train = transforms.Compose([
    #transfomss 안에
    ##randomCrop 랜덤하게 정해진 크리고 자름. 확률적으로 고양이 귀, 꼬리만 보고도 판단할 수 있게+ 밝기 조절도
    ##RandomHorizontalFlip> 좌우를 우좌로 자름
    
    transforms.RandomCrop(32, padding=4), # 4만큼의 padding을 부여한 후 32*32 random cropping
    transforms.RandomHorizontalFlip(), #0.5의 확률로 이미지 좌우 반전하여 넣어줌
    transforms.ToTensor(),
    transforms.Normalize((0,4914,0.4822,0.4465), (0.2023, 0.1994, 0.2010))
     # cifar10(0~1)의 r,g,b 채널 별 평균 및 분산으로 normalization
        ## => N(0,1)의 정규분포를 따르도록 만들어준후 input으로 넣어줍니다.    
])

# 데이터 전처리를 위한 코드
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    # cifar10(0~1)의 r,g,b 채널 별 평균 및 분산으로 normalization 
    ## =>N(0,1)의 정규분포를 따르도록 만들어준후 input으로 넣어줍니다.    
    
])


# 데이터 로딩
trainset = torchvision.datasets.CIFAR10(root='./data', train= True, download= True, transform=transform_train)
trainloader= torch.utils.data.DataLoader(trainset, batch_size= batch_size, shuffle= True, num_workers=1)

testset = torchvision.datasets.CIFAR10(root='./data', train= False, download= True, transform= transform_test)
testloader= torch.utils.data.DataLoader(testset, batch_size=100, shuffle= False, num_workers=1)
### @@질문- num_workers의 역할이 뭔가요?
#### 재은 답: 데이터를 로딩하면서 사용되는 서브 프로세스의 갯수를 지정해줍니다. 지정 안 하면 0으로 메인 프로세스에서 진행을 합니다. 
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [36]:
# 데이터 시각화
def showImaes(image, row):
    for _ in range(row):
        
        idx = np.random.choice(batch_size, 6) #0~127의 정수 중 6개를 임의로 선택
        imgaes = image.numpy()[idx].transpose(0,2,3,1).clip(0,1)  #선택된 index에 해당하는 이미지 가져옴
                 #선택된 index에 해당되는 이미지를 가져움
        plt.figure(figsize= (15,90)) #세로 길이: 15, 가로길이15*6의 화면 생성
        
        for i in range(161, 167):
            plt.subplot(i)
            plt.imshow(images[i - 161])
            plt.xticks([])
            plt.yticks([])
            
        plt.show()
for i , (image, labels) in enumerate(trainloader):
    showImages(image.squeeze(), 3)
    break

RuntimeError: ignored

In [32]:
fprint('[info] # of train batch :' , len(trainloader)) #트레인 로더 안에 배치 사이즈 있음.
## 데이터 사이즈를 배치 사이즈로 자르면 미니 배치의 수가 나옴. 따라서 이런 미내 배치가 몇개인지 알려줌
print('[info] # of test batch :' , len(testloader))

[info] # of train batch : 391
[info] # of test batch : 100


## 딥러닝 줄기 2개
 - (1) 트레인 코드: 테스트는 일단 제외하고
    - 인풋 받아서 포워딩을 함
    - 주요내용: 정답과 스코어링 및 학습 
 - (2) 모델 코드

### train_network(net, optimizer, tainloader, epoches)
- net
- optimizer, : 최적화 하기 위함
- trainloader : 데이터 트레이닝 하기 위해서 사용하는 X값들 그리고 Y 값들(X는 넣은것, Y는 이렇지 않을까 예측한 것)
- epochs : 반복 횟수

#### 옵티마이저에서 제로 그레드를 하는 이유:
    - 옵티마이저는 최적화를 하는것
    - 전진방향으로 돌고나서 백프로파게이션을 할때, 해당 값이 누적이 되어있으면 가중치가 너무나도 늘기 때문임.
    - 따라서 제로그레드를 통해 값들을 초기화 시켜야함

In [38]:
def train_network(net,optimizer, trainloader, epochs= 5):
    for epoch in range(epochs): #loop over the dataset multiple times
        
        running_loss= 0.0 #runnin loss를 저장하기 위한 변수입니다.
        for i, data in enumerate(trainloader): #한 Eopch만큼 돕니다. 
                                    ## 매 iteration마다 정해진 Batch sie만큼 데이터를 뱉습니다.
            # get the inputs
            inputs, labels = data #DataLoader iterator의 반환값은 input_data와 labesl의 튜플형식
            # 데이터 로더에는 x 에 대한 (train/test) 그리고 y 에대 한 (trani/test)가 있습니다.
            ## 여기서는 학습을 시킬 데이터 X를 inputs로 설정했고, 테스트용으로 쓸 Y를 labels로 표기
            inputs = inputs.to(device) #Pytorch에서 nn.Module에 넣어 Backprop을 계산하기 위해 
                                        ## gpu 연동을 이와 같이 해줘야합니다.
            labels = labels.to(device) #gpu에 올려줘요
            
            #zero the parameter gradients
            optimizer.zero_grad() #현재 기존의 backprop을 계산하기 위해 저장했던 activation buffer 비움
            
            
            #from+ backward+ optimize
            outputs = net(inputs) #인풋을 넣고 네트워크로부터 아웃풋 얻습니다. 
            ## **net이 모델 코드. 넷에서 인풋 받아서 아웃풋 뽑죠. 이 아웃풋과 정해진 로스를 구해요. 
            ###이 로스를 백프랍하고 옵티마이저 스탭
            
            loss = criterion(outputs, labels) #loss function 에 주어진 output을 얻어냅니다.
            # outputs는 모델에 넣어서 나온 값이고, labels는 trainloader 에서 y값들에 대한것(예측값)
            ##> 즉 이 2개에 대해서 loss가 얼마나 있는지를 criterion으로 체크게 하고
            loss.backward() #백워드를 진행하여 loss를 구함. 
            optimizer.step() #최적화 마쳤다. 이런 느낌
            
        # @@@@ 러닝로스 아래 코드들이 무슨 의미이지
            running_loss += loss.item() 
            if (i+1)%100 == 0:
                print('[%d %5d] loss : %3f' % (epoch +1, i+1, running_loss/100))
                running_loss = 0.0

print('Finished Training')

Finished Training


In [0]:
def test(model, test_loader):
    model.eval() 
    test_loss = 0
    correct = 0
    
    for data, target in test_loader:
        data= data.to(device)
        target= target.to(device) #기존의 train function의 데이터 처리부분과 같습니다.
        output= model(data)
        pred= output.max(1, keepdim= True)[1] # get the index of the max
        correct += pred.eq(target.view_as(pred)).sum().item() #정답 데이터의 